In [2]:
import pandas as pd

In [3]:
subset_pat = pd.read_json('../../Annotation/Patrick/batch2_patrick_result.json')
subset_thom = pd.read_json('../../Annotation/Thom/batch1_thom_result.json')
subset_chiel = pd.read_json('../../Annotation/Chiel/batch1_chiel_result.json')
subset_bar = pd.read_json('batch1_result.json')

In [25]:
subset_pat[subset_pat.vocop_match != 0]

,uuid,rubriek,notaris,inventarisNr,akteNr,akteType,datering,taal,beschrijving,namen,urls,text,name,data_matches,data_entry,vocop_match
2049,87d8b2f4-b586-8b52-5428-5b7fd24a6e8b,358,JAN VERLEIJ,11908,12813,Testament,1749-09-11,nederlands,"\nschip Bloemendaal, VOC kamer Amsterdam\n","[{'voornaam': 'Leendert', 'tussenvoegsel': Non...","['KLAB05596000423.JPG', 'KLAB05596000424.JPG',...",:15Calkoen Dit Testament vervat No: 437 geen f...,Leendert Zuijderland,"[{'index': 348698, 'name_original': 'Leendert ...","{'348698': {'ships': ['Bloemendaal'], 'rank': ...","[Leendert Suijderlandt, 348698]"
2071,e0d5cb34-cd51-5dae-cb7d-83aaf62f32a3,358,JAN VERLEIJ,11908,14377,Machtiging,1749-09-08,nederlands,\nschip Sloterdijk\n,"[{'voornaam': 'Jacobus', 'tussenvoegsel': None...","['KLAB05596000398.JPG', 'KLAB05596000399.JPG',...",No: 429 Procumatie gecasseerd den 8e: fept: 17...,Hermanus Soet,"[{'index': 174610, 'name_original': 'Harmanus ...","{'174610': {'ships': [], 'rank': [], 'location...","[Hermanus Soet, 374000]"
2282,b30c8a95-e5a2-f818-da9f-d19ec93d59e6,358,JAN VERLEIJ,11910,17760,Machtiging,1750-04-17,nederlands,"\nSchip de Ouderkerk, schip de Standvastigheid\n","[{'voornaam': 'Johannes', 'tussenvoegsel': 'va...","['KLAB05289000441.JPG', 'KLAB05289000442.JPG',...",No: 137 Procuratie gepasseert den 17e: april 1...,Jan Helt,"[{'index': 415, 'name_original': 'Jan Held', '...","{'415': {'ships': [], 'rank': [], 'location': ...","[Jan Helt, 417031]"


In [23]:
subset_bar[subset_bar.vocop_match != 0]

,uuid,rubriek,notaris,inventarisNr,akteNr,akteType,datering,taal,beschrijving,namen,urls,text,name,data_matches,data_entry,vocop_match
5668,16106777-6e18-ce1d-f4a0-650fb3247e3a,358,JAN VERLEIJ,11955,21595,Machtiging,1764-10-23,nederlands,"\nVOC schip Pijlswaert, schip De Drie Papagaij...","[{'voornaam': 'Jan', 'tussenvoegsel': None, 'a...","['KLAB06472000152.JPG', 'KLAB06472000153.JPG',...",No: 299 Procuratie gecasseerd den 23e: Octob: ...,Jan Hagenau,"[{'index': 88904, 'name_original': 'Jan Hagena...","{'88904': {'ships': [], 'rank': [], 'location'...","[Jan Hagenau, 457173]"
5687,73b36840-adfa-0db8-9d3f-e870cbe275d5,358,JAN VERLEIJ,11955,26890,Machtiging,1764-09-29,nederlands,"\nVOC schip Westerveld, innen van uitstaande g...","[{'voornaam': 'Evert', 'tussenvoegsel': None, ...","['KLAB06472000078.JPG', 'KLAB06472000079.JPG',...",No: 276 Procuratie gecasseerd den 29e: Sept: 1...,Elias Stodius,"[{'index': 476312, 'name_original': 'Elias Sto...","{'476312': {'ships': [], 'rank': [], 'location...","[Elias Stodius, 493407]"
5694,82e96f32-75e0-e937-5c7e-5a3e5c0a6792,358,JAN VERLEIJ,11955,27743,Machtiging,1764-11-05,nederlands,"\nVOC schip Cattendijk, schip De vrouw Elisabe...","[{'voornaam': 'Cornelis', 'tussenvoegsel': 'de...","['KLAB06472000201.JPG', 'KLAB06472000202.JPG',...",No: 314 Procuratie gecasseerd den 5e: navr: 17...,Fredrik Hove,"[{'index': 101246, 'name_original': 'Freedrik ...","{'101246': {'ships': ['Kattendijke', 'VROUWE E...","[Freedrik Hove, 101246]"
5846,934d3c19-5d20-33d8-5826-9f46658b64b2,358,JAN VERLEIJ,11957,28686,Machtiging,1765-07-01,nederlands,\nSchip Overschie; schip de Gouverneur Generaal\n,"[{'voornaam': 'Ernst August', 'tussenvoegsel':...","['KLAB06471000222.JPG', 'KLAB06471000223.JPG']",en odt Conden de hem dezeegeld den 2Jeaf 1755 ...,Ernst August Berkelman,"[{'index': 308820, 'name_original': 'Ernst Aug...","{'308820': {'ships': [], 'rank': [], 'location...","[Ernst Augustus Berkelman, 447428]"
5857,a517338b-8389-d6f7-18c8-29b1f8f944fc,358,JAN VERLEIJ,11957,29685,Machtiging,1765-07-06,nederlands,\nJoseph vertrok in 1761 aan boord van het VOC...,"[{'voornaam': 'Jan', 'tussenvoegsel': None, 'a...","['KLAB06471000251.JPG', 'KLAB06471000252.JPG',...",No: 195 Procuratie gecasseerd den 6e: Julij 17...,Jan Joseph,"[{'index': 5295, 'name_original': 'Jan Josephs...","{'5295': {'ships': [], 'rank': [], 'location':...","[Jan Joseph, 81104]"
5875,197025df-61b1-730c-efc4-a88fd46ec6fc,358,JAN VERLEIJ,11958,21795,Machtiging,1765-09-21,nederlands,\nSchip de Getrouwigheijd voor de Kamer Hoorn;...,"[{'voornaam': 'Willem', 'tussenvoegsel': None,...","['KLAB06470000093.JPG', 'KLAB06470000094.JPG',...",No: 292 Procuratie gecasseerd den 21e: Sept: 1...,Jan Wolter Visser,"[{'index': 442099, 'name_original': 'Jan Wolte...","{'442099': {'ships': ['Getrouwigheid', 'JERUSA...","[Jan Wolter Visser, 442099]"
5964,1bf4e148-14a4-dd4d-ef24-503a157766dc,358,JAN VERLEIJ,11960,21944,Machtiging,1766-08-25,nederlands,"\nVOC schip Damzigt, schip Walcheren, innen va...","[{'voornaam': 'Pieter', 'tussenvoegsel': 'van'...","['KLAB06468000354.JPG', 'KLAB06468000355.JPG',...",an No: 215 Procuratie gecasseerd den 25e: aug:...,Hendrik Dames,"[{'index': 55709, 'name_original': 'Hendrik Da...","{'55709': {'ships': [], 'rank': [], 'location'...","[Hendrik Dames, 80776]"
6001,c2a57380-363a-18cd-4a39-621fbc76345f,358,JAN VERLEIJ,11960,31329,Machtiging,1766-08-12,nederlands,\nKomber vertrok in 1765 aan boord van het VOC...,"[{'voornaam': 'Willem', 'tussenvoegsel': 'de',...","['KLAB06468000334.JPG', 'KLAB06468000335.JPG']",No: 208 Procuratie gecasseerd den 12e: Aug: 17...,Michiel Komber,"[{'index': 73206, 'name_original': 'Miechiel K...","{'73206': {'ships': ['Noord Nieuwland', 'SCHOO...","[Miechiel Komber, 73206]"
6044,4a742b73-d2fb-cf56-9d0e-1195882de934,358,JAN VERLEIJ,11961,24564,Machtiging,1766-09-30,nederlands,"\nVOC schip Pallas, machtiging aan zijn vrouw\n","[{'voornaam': 'Jacob', 'tussenvoegsel': None, ...","['KLAB06467000112.JPG', 'KLAB064

In [4]:
subset_thom[subset_thom.vocop_match != 0]

,uuid,rubriek,notaris,inventarisNr,akteNr,akteType,datering,taal,beschrijving,namen,urls,text,name,data_matches,data_entry,vocop_match
840,8ed577d5-dc59-e08a-683a-e57f47ee126e,358,JAN VERLEIJ,11895,14505,Machtiging,1745-10-01,nederlands,"\nIn 1742 met 't schip ""'t Fortuijn"" als schie...","[{'voornaam': 'Jan', 'tussenvoegsel': 'van', '...","['KLAB05405000003.JPG', 'KLAB05405000004.JPG',...",Dn 4buud No: 457 Procuratie gepasseert den Eer...,Jan van Zee,"[{'index': 108733, 'name_original': 'Jan van Z...","{'108733': {'ships': [], 'rank': [], 'location...","[Jan van Zee, 277287]"
1024,2179f9a4-6aff-a279-27b5-8637e2d71d67,358,JAN VERLEIJ,11898,17028,Machtiging,1746-08-06,nederlands,\nSchip de Brouwer; Schip Spaanderswoud\n,"[{'voornaam': 'Claas', 'tussenvoegsel': None, ...","['KLAB05443000151.JPG', 'KLAB05443000152.JPG']",No: 366 Procuratie gepasseert den 6e: Augustus...,Jan Haveman,"[{'index': 95444, 'name_original': 'Jan Haman'...","{'95444': {'ships': [], 'rank': [], 'location'...","[Jan Haveman, 310678]"
1032,df3ef34a-a719-033c-0f12-0f719913b3ac,358,JAN VERLEIJ,11898,17356,Obligatie,1746-09-29,nederlands,"\nin dienst OIC met schip ""'t huijs te Manpad""\n","[{'voornaam': 'Fredrik', 'tussenvoegsel': None...","['KLAB05443000454.JPG', 'KLAB05443000455.JPG',...",No: 461 Oblegdee gepasseert den 29 Sept: 17461...,Hendrik de Lange,"[{'index': 48086, 'name_original': 'Hendrik La...","{'48086': {'ships': [], 'rank': [], 'location'...","[Hendrik de Lange, 329199]"
1131,b1867805-a5e1-7adb-6c53-436c6c7703f2,358,JAN VERLEIJ,11899,17638,Machtiging,1746-12-09,nederlands,\nschip Weltevreden\n,"[{'voornaam': 'Sijmen', 'tussenvoegsel': 'van'...","['KLAB05532000409.JPG', 'KLAB05532000410.JPG',...",N55 Presnentie gecasseerd den 9 Dacomber 1746O...,Pieter Schrick,"[{'index': 67453, 'name_original': 'Pieter Sch...","{'67453': {'ships': [], 'rank': [], 'location'...","[Pieter Schriek, 327723]"
1212,6eeff486-d429-f2ee-40f6-46bdfda3cbd6,358,JAN VERLEIJ,11900,19676,Obligatie,1747-04-24,nederlands,"\nschip het Kasteel van TIlburg, VOC kamer Ams...","[{'voornaam': 'Philip Lodewijk', 'tussenvoegse...","['KLAB05523000511.JPG', 'KLAB05523000512.JPG',...",37 : 15No: 160 Sbteijate gepasseert den 24e: a...,Abraham Swart,"[{'index': 72145, 'name_original': 'Abraham Sw...","{'72145': {'ships': [], 'rank': [], 'location'...","[Abraham Swart, 338951]"
1321,a1cb3cb1-12d2-8848-b17d-d736777e0ad6,358,JAN VERLEIJ,11902,65696,Machtiging,1747-09-22,nederlands,"\nschip Stadwijk, VOC kamer Amsterdam; schip D...","[{'voornaam': 'Johan Hendrik', 'tussenvoegsel'...","['KLAB05517000096.JPG', 'KLAB05517000097.JPG']",No: 390 Procuratie Grede den 22e: Feptem ber 1...,Johan Hendrik Haase,"[{'index': 80985, 'name_original': 'Johan Hend...","{'80985': {'ships': [], 'rank': [], 'location'...","[Johan Hendrik Haze, 259917]"


In [21]:
subset_chiel[subset_chiel.vocop_match != 0].iloc[7].text

"No: 333 Procuratie gecasseerd den 7e: Sept: 1348EOp heden den 7: Aepril des Jaars 1755 Compareerden Voor mij Jan Ver Notaris publiek bij den Ed: hole van hol Geadmitteert binnen Amsterdam Resider DE: Jans27e: Sipt 748 gCompareerden d' E. Jan Christoffel Helleff van Coppenhagen in 't Jaer 1746 voor bootsmans maat met het Schip Spaander wont voor de kamer Zeeland na Oostindien uijtgevaren en nu onlangs met 't Selve Schip in Gr: als onderstuurman voor de kamer Amsterdam gerepatrieert en behoorlijk affgedankt 5: de 1e: 0e: n & present binne dese Stadd De wel Verklaarde bij desen te Constiuueeren & magtieg te maken d' E. Dirk van den Ende wonende alhier omme in den naam & Van Weegens de Comparant te eijsschen Vorderen & ontfangen Van d' Ed: Heere Bewind hebbenen van de oostind: Compagnie ter kamere Amsterdam Sijn Comparants kist twee flessen kelders en twee potten alle gemerkt L: C en nog J: C6. mitsgaders nog om te verzoeken & ontfangen Sijn Comparants affreekening wegens voornoemde diens

In [16]:
subset_chiel[subset_chiel.vocop_match != 0]

,uuid,rubriek,notaris,inventarisNr,akteNr,akteType,datering,taal,beschrijving,namen,urls,text,name,data_matches,data_entry,vocop_match
1399,4d0a5598-ea7b-7040-e053-b784100a6264,358,JAN VERLEIJ,11903,59238,Testament,1748-01-06,nederlands,None,"[{'voornaam': 'Coenraed Hendrik', 'tussenvoegs...","['KLAB05521000021.JPG', 'KLAB05521000022.JPG',...",No: 6 Testament gepasseert e den 6e: Januarij ...,Coenraed Hendrik Vink,"[{'index': 275177, 'name_original': 'Coenraad ...","{'275177': {'ships': [], 'rank': [], 'location...","[Coenraad Hendrik Vinke, 275177]"
1415,4d0a5599-7775-7040-e053-b784100a6264,358,JAN VERLEIJ,11903,64925,Machtiging,1748-01-15,nederlands,None,"[{'voornaam': 'Claas', 'tussenvoegsel': None, ...","['KLAB05521000061.JPG', 'KLAB05521000062.JPG']",L: -N: 18 Procuratie gepasseert e den 15e: Jan...,Claas Pietersz,"[{'index': 3754, 'name_original': 'Claas Piete...","{'3754': {'ships': ['nan'], 'rank': [], 'locat...","[Claas Pietersz, 326827]"
1416,4d0a5599-7775-7040-e053-b784100a6264,358,JAN VERLEIJ,11903,64925,Machtiging,1748-01-15,nederlands,None,"[{'voornaam': 'Claas', 'tussenvoegsel': None, ...","['KLAB05521000061.JPG', 'KLAB05521000062.JPG']",L: -N: 18 Procuratie gepasseert e den 15e: Jan...,Christiaan Andriesz,"[{'index': 949, 'name_original': 'Christiaan A...","{'949': {'ships': ['nan'], 'rank': [], 'locati...","[Christiaan Andriesz, 292302]"
1463,3cb4f4c9-3056-3b97-c26f-7d010de31eae,358,JAN VERLEIJ,11903,75050,Attestatie,1748-02-10,nederlands,\nVerklaren familierelaties te kennen.\n,"[{'voornaam': 'Pieter Cornelis', 'tussenvoegse...","['KLAB05521000125.JPG', 'KLAB05521000126.JPG',...",No: 38 Verklaring gepasseert den 10e: februari...,Jan de Bruijn,"[{'index': 32, 'name_original': 'Jan de Bruijn...","{'32': {'ships': [], 'rank': [], 'location': [...","[Jan de Bruijn, 288967]"
1728,574e9081-263d-c1a6-db55-c1cb55e9a434,358,JAN VERLEIJ,11905,13130,Machtiging,1748-09-19,nederlands,"\nschip de Eendragt, VOC kamer Amsterdam; schi...","[{'voornaam': 'Jan', 'tussenvoegsel': None, 'a...","['KLAB05559000104.JPG', 'KLAB05559000105.JPG',...",No: 360 Crocaratie gecasseerd den 19e: Sept: 1...,Jan Schouten,"[{'index': 7034, 'name_original': 'Jan Schout'...","{'7034': {'ships': [], 'rank': [], 'location':...","[Jan Schoutte, 297792]"
1729,574e9081-263d-c1a6-db55-c1cb55e9a434,358,JAN VERLEIJ,11905,13130,Machtiging,1748-09-19,nederlands,"\nschip de Eendragt, VOC kamer Amsterdam; schi...","[{'voornaam': 'Jan', 'tussenvoegsel': None, 'a...","['KLAB05559000104.JPG', 'KLAB05559000105.JPG',...",No: 360 Crocaratie gecasseerd den 19e: Sept: 1...,Hendrik Nieuboer,"[{'index': 49105, 'name_original': 'Hendrik Ne...","{'49105': {'ships': [], 'rank': [], 'location'...","[Hendrik Nieuwboer, 142291]"
1822,b011937e-a96a-8ce6-5856-37538debf093,358,JAN VERLEIJ,11905,18536,Machtiging,1748-11-29,nederlands,\nVoor Kamer Amsterdam met schip Haerlmen naar...,"[{'voornaam': 'Catharina', 'tussenvoegsel': No...","['KLAB05559000570.JPG', 'KLAB05559000571.JPG',...",No: 507 Procuratie gepuisseert den 29e: Novemb...,Jacobus de Heer,"[{'index': 7410, 'name_original': 'Jacobus de ...","{'7410': {'ships': [], 'rank': [], 'location':...","[Jacobus de Heere, 376541]"
1823,6d3a1c09-5c6f-ce55-77ae-d618e1c9d0f7,358,JAN VERLEIJ,11905,18615,Machtiging,1748-09-07,nederlands,None,"[{'voornaam': 'Abraham', 'tussenvoegsel': 'van...","['KLAB05559000020.JPG', 'KLAB05559000021.JPG',...",No: 333 Procuratie gecasseerd den 7e: Sept: 13...,Abraham van den Heuvel,"[{'index': 83203, 'name_original': 'Abraham va...","{'83203': {'ships': [], 'rank': [], 'location'...","[Abraham van der Heuvel, 125364]"
1825,6d3a1c09-5c6f-ce55-77ae-d618e1c9d0f7,358,JAN VERLEIJ,11905,18615,Machtiging,1748-09-07,nederlands,None,"[{'voornaam': 'Abraham', 'tussenvoegsel': 'van...","['KLAB05559000020.JPG', 'KLAB05559000021.JPG',...",No: 333 Procuratie gecasseerd den 7e: Sept: 13...,Pieter van Essen,"[{'index': 20310, 'name_original': 'Pieter van...","{'20310': {'ships': [], 'rank': [], 'location'...","[Piete

#### Correcties:
Chiel:
- Christiaan Andriesz 1416
- Jan de Bruijn 1463
- Abraham van de Heuvel (1823 super interessant twijfel geval)
- Pieter van Essen (1825 super interessant twijfel geval)

Barry:
- Dirk Huijer 6044, mogelijke match met Dirk Huijer 76412